# Tutorial 4: Token Standards (ERC-20, ERC-721)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/blockchain-tokens-and-incentives/blob/main/notebooks/tutorial_04_token_standards.ipynb)

---

> *"A token is not money. A token is a state entry in a smart contract. Understanding this distinction is the difference between speculation and engineering."*
>
> — Brenn Auster

---

## What Tokens Actually Are

Before we can implement Auster's staking mechanism, we need the building blocks: **tokens**. In popular discourse, tokens are "cryptocurrencies" or "digital assets." In engineering, a token is far simpler: it's a **state entry in a smart contract** — a row in a ledger that records who owns what.

An ERC-20 token is a contract with a mapping: `address → balance`. That's it. Transfer means decrement one balance and increment another. The entire complexity of DeFi — billions of dollars of value — rests on this simple data structure.

This tutorial implements both major token standards from scratch in Python, then deploys guild tokens for the Traders Guild economy.

## Learning Objectives

In this tutorial, you will:

1. **Understand tokens as state** — balances in a mapping, not physical objects
2. **Implement ERC-20** — the fungible token standard (guild trade tokens)
3. **Implement ERC-721** — the non-fungible token standard (manuscript certificates)
4. **Deploy guild tokens** — mint, transfer, approve, and burn tokens
5. **Connect to mechanism design** — tokens as the substrate for incentive systems

**Technical Concepts:**
- ERC-20: fungible tokens (every token identical)
- ERC-721: non-fungible tokens (every token unique)
- Token operations: mint, transfer, approve, transferFrom, burn
- The allowance pattern and why it exists

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

scholars = pd.read_csv(BASE_URL + "scholars.csv")
guild_trades = pd.read_csv(BASE_URL + "guild_trade_ledger.csv")

print(f"Scholars: {len(scholars)} (token holders)")
print(f"Guild trades: {len(guild_trades)} (transaction history)")

## ERC-20: The Fungible Token

ERC-20 defines the interface for **fungible** tokens — tokens where every unit is identical and interchangeable, like coins in a currency. The standard specifies six functions and two events.

In [ ]:
class ERC20:
    """Python implementation of the ERC-20 fungible token standard."""
    
    def __init__(self, name, symbol, initial_supply, deployer):
        self.name = name
        self.symbol = symbol
        self.decimals = 18
        self._balances = defaultdict(int)
        self._allowances = defaultdict(lambda: defaultdict(int))
        self._total_supply = 0
        self._event_log = []
        
        # Mint initial supply to deployer
        self._mint(deployer, initial_supply)
    
    def _mint(self, to, amount):
        self._balances[to] += amount
        self._total_supply += amount
        self._log('Mint', {'to': to, 'amount': amount})
    
    def _burn(self, holder, amount):
        assert self._balances[holder] >= amount, f"Insufficient balance: {holder}"
        self._balances[holder] -= amount
        self._total_supply -= amount
        self._log('Burn', {'from': holder, 'amount': amount})
    
    def total_supply(self):
        return self._total_supply
    
    def balance_of(self, account):
        return self._balances[account]
    
    def transfer(self, sender, to, amount):
        assert self._balances[sender] >= amount, f"Insufficient balance: {sender}"
        self._balances[sender] -= amount
        self._balances[to] += amount
        self._log('Transfer', {'from': sender, 'to': to, 'amount': amount})
        return True
    
    def approve(self, owner, spender, amount):
        self._allowances[owner][spender] = amount
        self._log('Approval', {'owner': owner, 'spender': spender, 'amount': amount})
        return True
    
    def allowance(self, owner, spender):
        return self._allowances[owner][spender]
    
    def transfer_from(self, spender, from_addr, to, amount):
        assert self._allowances[from_addr][spender] >= amount, "Insufficient allowance"
        assert self._balances[from_addr] >= amount, "Insufficient balance"
        self._allowances[from_addr][spender] -= amount
        self._balances[from_addr] -= amount
        self._balances[to] += amount
        self._log('Transfer', {'from': from_addr, 'to': to, 'amount': amount,
                               'spender': spender})
        return True
    
    def _log(self, event_type, data):
        self._event_log.append({'event': event_type, **data})
    
    def get_events(self):
        return pd.DataFrame(self._event_log)

print("ERC-20 standard functions:")
print("  totalSupply()              → total tokens in existence")
print("  balanceOf(account)         → token balance of account")
print("  transfer(to, amount)       → send tokens from caller")
print("  approve(spender, amount)   → allow spender to use your tokens")
print("  allowance(owner, spender)  → remaining approved amount")
print("  transferFrom(from, to, amount) → spend approved tokens")

## Deploying the Guild Trade Token

Let's deploy the Traders Guild's fungible token: **GuildCoin (GLD)**. This will serve as the currency for all guild trade operations.

In [ ]:
# Deploy GuildCoin
GUILD_TREASURY = "Traders Guild Treasury"
INITIAL_SUPPLY = 1_000_000  # 1 million GLD

gld = ERC20("GuildCoin", "GLD", INITIAL_SUPPLY, GUILD_TREASURY)

print(f"Token deployed: {gld.name} ({gld.symbol})")
print(f"Total supply: {gld.total_supply():,} {gld.symbol}")
print(f"Treasury balance: {gld.balance_of(GUILD_TREASURY):,} {gld.symbol}")

In [ ]:
# Distribute tokens to guild members based on their trade volume
trader_volume = guild_trades.groupby('seller')['total_value'].sum()
top_traders = trader_volume.sort_values(ascending=False).head(15)

# Allocate proportional to trade volume (10% of supply)
allocation_pool = INITIAL_SUPPLY * 0.10
total_volume = top_traders.sum()

print("Token distribution to top 15 traders:\n")
for trader, volume in top_traders.items():
    allocation = int(allocation_pool * volume / total_volume)
    gld.transfer(GUILD_TREASURY, trader, allocation)
    print(f"  {trader:20s}: {allocation:8,d} GLD (volume share: {volume/total_volume:.1%})")

print(f"\nTreasury remaining: {gld.balance_of(GUILD_TREASURY):,} GLD")
print(f"Distributed: {INITIAL_SUPPLY - gld.balance_of(GUILD_TREASURY):,} GLD")

In [ ]:
# Demonstrate the allowance pattern: 
# Brenn Auster wants to let the staking contract spend his tokens
STAKING_CONTRACT = "Guild Staking Contract"

# Step 1: Auster approves the staking contract
auster_balance = gld.balance_of('Brenn Auster')
stake_amount = auster_balance // 2  # Stake half

gld.approve('Brenn Auster', STAKING_CONTRACT, stake_amount)
print(f"Auster balance: {auster_balance:,} GLD")
print(f"Auster approved staking contract for: {stake_amount:,} GLD")
print(f"Current allowance: {gld.allowance('Brenn Auster', STAKING_CONTRACT):,} GLD")

# Step 2: Staking contract pulls the tokens
gld.transfer_from(STAKING_CONTRACT, 'Brenn Auster', STAKING_CONTRACT, stake_amount)
print(f"\nAfter staking:")
print(f"  Auster balance: {gld.balance_of('Brenn Auster'):,} GLD")
print(f"  Staking contract: {gld.balance_of(STAKING_CONTRACT):,} GLD")
print(f"  Remaining allowance: {gld.allowance('Brenn Auster', STAKING_CONTRACT):,} GLD")

print(f"\nThe allowance pattern: approve first, then transferFrom.")
print(f"This two-step process prevents contracts from taking tokens without permission.")

## ERC-721: The Non-Fungible Token

ERC-721 defines the interface for **non-fungible** tokens — tokens where each is unique. In Densworld, these represent manuscript authenticity certificates: each manuscript gets a unique token proving its provenance.

In [ ]:
class ERC721:
    """Python implementation of the ERC-721 non-fungible token standard."""
    
    def __init__(self, name, symbol):
        self.name = name
        self.symbol = symbol
        self._owners = {}        # token_id → owner
        self._balances = defaultdict(int)  # owner → count
        self._token_approvals = {}  # token_id → approved address
        self._operator_approvals = defaultdict(lambda: defaultdict(bool))
        self._token_metadata = {}  # token_id → metadata dict
        self._next_token_id = 1
        self._event_log = []
    
    def mint(self, to, metadata=None):
        token_id = self._next_token_id
        self._next_token_id += 1
        self._owners[token_id] = to
        self._balances[to] += 1
        if metadata:
            self._token_metadata[token_id] = metadata
        self._log('Mint', {'to': to, 'token_id': token_id})
        return token_id
    
    def owner_of(self, token_id):
        assert token_id in self._owners, f"Token {token_id} does not exist"
        return self._owners[token_id]
    
    def balance_of(self, owner):
        return self._balances[owner]
    
    def transfer(self, sender, to, token_id):
        assert self._owners[token_id] == sender, f"Not owner of token {token_id}"
        self._owners[token_id] = to
        self._balances[sender] -= 1
        self._balances[to] += 1
        self._token_approvals.pop(token_id, None)
        self._log('Transfer', {'from': sender, 'to': to, 'token_id': token_id})
    
    def approve(self, owner, approved, token_id):
        assert self._owners[token_id] == owner, "Not owner"
        self._token_approvals[token_id] = approved
        self._log('Approval', {'owner': owner, 'approved': approved, 'token_id': token_id})
    
    def get_approved(self, token_id):
        return self._token_approvals.get(token_id, None)
    
    def token_metadata(self, token_id):
        return self._token_metadata.get(token_id, {})
    
    def total_supply(self):
        return len(self._owners)
    
    def _log(self, event_type, data):
        self._event_log.append({'event': event_type, **data})
    
    def get_events(self):
        return pd.DataFrame(self._event_log)

print("ERC-721 standard functions:")
print("  balanceOf(owner)         → number of NFTs owned")
print("  ownerOf(tokenId)         → owner of specific NFT")
print("  transferFrom(from,to,id) → transfer specific NFT")
print("  approve(to, tokenId)     → approve specific NFT transfer")
print("  getApproved(tokenId)     → who is approved for this NFT")

## Deploying Manuscript Certificates

Each authenticated Densworld manuscript gets a unique NFT — a **Manuscript Certificate (MSC)**. This connects to Course 04's digital signature work: the NFT is the on-chain record of Serath Kyne's signature verification.

In [ ]:
# Deploy Manuscript Certificate NFT
msc = ERC721("ManuscriptCertificate", "MSC")

# Mint certificates for scholars based on their major_works
# Each scholar's manuscripts get an NFT
minted = []
for _, scholar in scholars.head(15).iterrows():
    works = str(scholar.get('major_works', ''))
    if works and works != 'nan':
        ms_ids = [w.strip() for w in works.split(',') if w.strip()]
        for ms_id in ms_ids[:3]:  # Max 3 per scholar for demo
            token_id = msc.mint(scholar['name'], metadata={
                'manuscript_id': ms_id,
                'author': scholar['name'],
                'specialty': scholar['specialty'],
                'school': scholar['philosophical_school'],
                'authenticated': True
            })
            minted.append({
                'token_id': token_id,
                'owner': scholar['name'],
                'manuscript': ms_id
            })

print(f"Minted {msc.total_supply()} Manuscript Certificates")
print(f"\nCertificate registry:")
for m in minted[:10]:
    print(f"  MSC #{m['token_id']:3d} → {m['owner']:20s} ({m['manuscript']})")

In [ ]:
# NFT ownership distribution
owner_counts = defaultdict(int)
for m in minted:
    owner_counts[m['owner']] += 1

owners_df = pd.DataFrame([
    {'owner': k, 'nfts': v} for k, v in sorted(owner_counts.items(), key=lambda x: -x[1])
])

plt.figure(figsize=(10, 5))
plt.barh(owners_df['owner'], owners_df['nfts'], color='steelblue', alpha=0.8, edgecolor='black')
plt.xlabel('Number of Manuscript Certificates')
plt.title('NFT Distribution: Manuscript Certificates by Scholar')
plt.tight_layout()
plt.show()

In [ ]:
# Demonstrate NFT transfer and metadata
# Grigsu's manuscript being transferred (sale, inheritance, or archive deposit)
grigsu_tokens = [m for m in minted if m['owner'] == 'Grigsu Haldo']
if grigsu_tokens:
    token = grigsu_tokens[0]
    print(f"Before transfer:")
    print(f"  MSC #{token['token_id']} owner: {msc.owner_of(token['token_id'])}")
    print(f"  Metadata: {msc.token_metadata(token['token_id'])}")
    
    # Transfer to the Capital Archive
    msc.transfer('Grigsu Haldo', 'Capital Archive', token['token_id'])
    
    print(f"\nAfter transfer to Capital Archive:")
    print(f"  MSC #{token['token_id']} owner: {msc.owner_of(token['token_id'])}")
    print(f"  Grigsu's remaining certificates: {msc.balance_of('Grigsu Haldo')}")
    print(f"  Capital Archive certificates: {msc.balance_of('Capital Archive')}")
    print(f"\nThe metadata persists — ownership changes, provenance doesn't.")

## Comparing ERC-20 and ERC-721

The two standards represent fundamentally different approaches to ownership.

In [ ]:
# Side-by-side comparison
print(f"{'Property':30s} {'ERC-20 (GuildCoin)':25s} {'ERC-721 (ManuscriptCert)':25s}")
print(f"{'─'*80}")
print(f"{'Fungibility':30s} {'Fungible (1 GLD = 1 GLD)':25s} {'Non-fungible (each unique)':25s}")
print(f"{'State structure':30s} {'address → balance':25s} {'tokenId → owner':25s}")
print(f"{'Transfer granularity':30s} {'Any amount':25s} {'Whole token only':25s}")
print(f"{'Metadata':30s} {'None (all identical)':25s} {'Per-token (unique data)':25s}")
print(f"{'Total supply':30s} {str(gld.total_supply()) + ' GLD':25s} {str(msc.total_supply()) + ' MSC':25s}")
print(f"{'Divisible':30s} {'Yes (18 decimals)':25s} {'No':25s}")
print(f"{'Use case':30s} {'Currency, staking':25s} {'Certificates, identity':25s}")

print(f"\nIn the guild economy:")
print(f"  GLD = money (what you trade with)")
print(f"  MSC = identity (what proves your work)")

## Token Economics: Supply and Distribution

Token design isn't just about the standard — it's about the **economics**. How tokens are distributed determines who has power in the system.

In [ ]:
# Analyze token distribution
all_holders = {}
for m in minted:
    holder = msc.owner_of(m['token_id'])  # Current owner (may have changed)
    all_holders[holder] = all_holders.get(holder, 0)

# GLD distribution
gld_holders = {}
events = gld.get_events()
for addr in set(events.get('to', [])) | set(events.get('from', [])):
    if isinstance(addr, str):
        bal = gld.balance_of(addr)
        if bal > 0:
            gld_holders[addr] = bal

gld_df = pd.DataFrame([
    {'holder': k, 'balance': v} for k, v in 
    sorted(gld_holders.items(), key=lambda x: -x[1])
])

# Compute Gini coefficient (measure of inequality)
balances = sorted(gld_df['balance'].values)
n = len(balances)
if n > 0:
    cumulative = np.cumsum(balances)
    gini = (2 * np.sum((np.arange(1, n+1) * balances))) / (n * np.sum(balances)) - (n + 1) / n
else:
    gini = 0

print(f"GLD Token Distribution Analysis:")
print(f"  Total holders: {len(gld_df)}")
print(f"  Total supply: {gld.total_supply():,}")
print(f"  Gini coefficient: {gini:.3f} (0=equal, 1=concentrated)")
print(f"\n  Top 5 holders:")
for _, row in gld_df.head(5).iterrows():
    share = row['balance'] / gld.total_supply() * 100
    print(f"    {row['holder']:25s} {row['balance']:>10,d} GLD ({share:5.1f}%)")

In [ ]:
# Lorenz curve (inequality visualization)
if len(balances) > 0:
    total = sum(balances)
    lorenz = np.cumsum(balances) / total
    lorenz = np.insert(lorenz, 0, 0)
    pop_pct = np.linspace(0, 1, len(lorenz))
    
    plt.figure(figsize=(8, 8))
    plt.plot(pop_pct, lorenz, 'b-', linewidth=2, label=f'GLD distribution (Gini={gini:.3f})')
    plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Perfect equality')
    plt.fill_between(pop_pct, lorenz, pop_pct, alpha=0.1, color='blue')
    plt.xlabel('Cumulative Share of Holders')
    plt.ylabel('Cumulative Share of Tokens')
    plt.title('Lorenz Curve: GuildCoin Distribution')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

print(f"The treasury holds {gld.balance_of(GUILD_TREASURY)/gld.total_supply():.1%} of supply.")
print(f"In governance (Course 06), this determines voting power.")
print(f"In staking (Tutorial 05), this determines validation weight.")

## Token Operations: The Full Lifecycle

Let's simulate a series of guild economy operations to see both token standards in action.

In [ ]:
# Simulate guild economy operations
print("=== GUILD ECONOMY SIMULATION ===")
print()

# 1. Trade: Auster buys iron_ore from Torren Gael
trade_price = 500
if gld.balance_of('Brenn Auster') >= trade_price:
    gld.transfer('Brenn Auster', 'Torren Gael', trade_price)
    print(f"1. TRADE: Auster pays Gael {trade_price} GLD for iron_ore")
    print(f"   Auster: {gld.balance_of('Brenn Auster'):,} GLD | Gael: {gld.balance_of('Torren Gael'):,} GLD")

# 2. Staking: Gael stakes tokens in the staking contract
gael_stake = gld.balance_of('Torren Gael') // 3
gld.approve('Torren Gael', STAKING_CONTRACT, gael_stake)
gld.transfer_from(STAKING_CONTRACT, 'Torren Gael', STAKING_CONTRACT, gael_stake)
print(f"\n2. STAKE: Gael stakes {gael_stake:,} GLD")
print(f"   Staking contract: {gld.balance_of(STAKING_CONTRACT):,} GLD")

# 3. Mint manuscript certificate for a new work
new_cert = msc.mint('Brenn Auster', metadata={
    'manuscript_id': 'MS-NEW-001',
    'title': 'Self-Enforcing Agreements',
    'year': 867,
    'authenticated': True
})
print(f"\n3. MINT: Auster receives MSC #{new_cert} for 'Self-Enforcing Agreements'")

# 4. Burn tokens (treasury buyback)
burn_amount = 10000
gld._burn(GUILD_TREASURY, burn_amount)
print(f"\n4. BURN: Treasury burns {burn_amount:,} GLD (deflationary mechanism)")
print(f"   New total supply: {gld.total_supply():,} GLD")

print(f"\n=== Event Log ===")
events = gld.get_events()
print(f"Total GLD events: {len(events)}")
print(events['event'].value_counts().to_string())

## Connecting to the Guild Economy

The token standards are the building blocks. Here's how they compose into the full guild economy that Tutorial 08 (capstone) will build.

In [ ]:
# The five token types in the guild economy
token_types = [
    ('GuildCoin (GLD)', 'ERC-20', 'Fungible trade currency', 'Tutorial 04'),
    ('ManuscriptCert (MSC)', 'ERC-721', 'Manuscript authenticity NFTs', 'Tutorial 04'),
    ('StakedGLD (sGLD)', 'ERC-20', 'Receipt for staked GLD', 'Tutorial 05'),
    ('LP Tokens', 'ERC-20', 'Liquidity pool shares', 'Tutorial 06'),
    ('Reputation (REP)', 'ERC-20', 'Non-transferable reputation', 'Tutorial 08'),
]

print("The Guild Economy Token System:")
print(f"{'Token':25s} {'Standard':10s} {'Purpose':40s} {'Introduced':12s}")
for name, std, purpose, tut in token_types:
    print(f"{name:25s} {std:10s} {purpose:40s} {tut:12s}")

print(f"\nTokens compose: stake GLD → get sGLD → provide liquidity → get LP tokens")
print(f"Each layer adds functionality while maintaining composability.")

## Exercises

### Exercise 1: Token Velocity

Compute the **token velocity** — total transfer volume divided by supply — for GLD using the guild trade ledger. High velocity means tokens change hands quickly (good for a currency). Low velocity means tokens are hoarded (good for a store of value).

In [ ]:
# Simulate token velocity from guild trade data
total_transfer_volume = guild_trades['total_value'].sum()
supply = gld.total_supply()
velocity = total_transfer_volume / supply

print(f"Token Velocity Analysis:")
print(f"  Total trade volume: {total_transfer_volume:,.0f}")
print(f"  Token supply: {supply:,}")
print(f"  Velocity: {velocity:.2f}x per period")
print(f"\n  Velocity > 1: tokens circulate actively (medium of exchange)")
print(f"  Velocity < 1: tokens are held (store of value)")
print(f"\n  GLD velocity: {'high (currency)' if velocity > 1 else 'moderate' if velocity > 0.5 else 'low (store of value)'}")

### Exercise 2: ERC-1155 Multi-Token

ERC-1155 combines ERC-20 and ERC-721 into one contract — it can hold both fungible and non-fungible tokens. Sketch an implementation with a single `balanceOf(account, id)` function.

In [ ]:
class ERC1155:
    """Simplified ERC-1155 multi-token standard."""
    def __init__(self):
        # (account, token_id) → balance
        self._balances = defaultdict(lambda: defaultdict(int))
        self._token_supply = defaultdict(int)
    
    def mint(self, to, token_id, amount):
        self._balances[to][token_id] += amount
        self._token_supply[token_id] += amount
    
    def balance_of(self, account, token_id):
        return self._balances[account][token_id]
    
    def transfer(self, sender, to, token_id, amount):
        assert self._balances[sender][token_id] >= amount
        self._balances[sender][token_id] -= amount
        self._balances[to][token_id] += amount

# Demo: guild commodity tokens (fungible) + certificates (NFT, amount=1)
multi = ERC1155()

# Fungible commodity tokens
IRON_TOKEN_ID = 100
VELLUM_TOKEN_ID = 101
multi.mint('Torren Gael', IRON_TOKEN_ID, 500)
multi.mint('Brenn Auster', VELLUM_TOKEN_ID, 200)

# NFT certificates (amount = 1 each)
CERT_BASE = 1000
multi.mint('Grigsu Haldo', CERT_BASE + 1, 1)  # Unique cert
multi.mint('Yasho Krent', CERT_BASE + 2, 1)

print(f"ERC-1155 Multi-Token Contract:")
print(f"  Gael's iron tokens: {multi.balance_of('Torren Gael', IRON_TOKEN_ID)}")
print(f"  Auster's vellum tokens: {multi.balance_of('Brenn Auster', VELLUM_TOKEN_ID)}")
print(f"  Grigsu's cert #{CERT_BASE+1}: {multi.balance_of('Grigsu Haldo', CERT_BASE+1)} (NFT)")
print(f"\nOne contract handles both fungible and non-fungible tokens.")

### Exercise 3: Whale Analysis

A "whale" holds more than 5% of token supply. How many whales exist in the GLD distribution? What share of total supply do they control?

In [ ]:
whale_threshold = gld.total_supply() * 0.05
whales = gld_df[gld_df['balance'] >= whale_threshold]

print(f"Whale analysis (threshold: {whale_threshold:,.0f} GLD = 5% of supply):")
print(f"  Whales: {len(whales)}")
whale_total = whales['balance'].sum()
print(f"  Whale holdings: {whale_total:,.0f} GLD ({whale_total/gld.total_supply():.1%} of supply)")
print(f"\n  Whale list:")
for _, w in whales.iterrows():
    print(f"    {w['holder']:25s} {w['balance']:>10,d} GLD ({w['balance']/gld.total_supply():.1%})")
print(f"\nWhale concentration has governance implications (Course 06):")
print(f"token-weighted voting gives whales disproportionate power.")

## Summary

In this tutorial, we learned:

1. **Tokens are state entries** — a mapping from address to balance, stored in a smart contract
2. **ERC-20** implements fungible tokens — every GLD is identical, divisible, and interchangeable
3. **ERC-721** implements non-fungible tokens — each MSC is unique with its own metadata
4. **The allowance pattern** enables smart contract interactions without surrendering custody
5. **Token distribution** determines system power dynamics — Gini coefficients and whale analysis matter

**Key insight:** Tokens are the building blocks of mechanism design on blockchain. ERC-20 enables currency, staking, and liquidity. ERC-721 enables identity, provenance, and certification. Together, they provide the substrate for Auster's incentive framework.

---

**Next Tutorial:** Staking and Slashing — lock up tokens to participate, lose them for misbehavior.

---

> *"A token is a claim. A staked token is a promise. A slashed token is a broken promise made visible."*
>
> — Brenn Auster